In [1]:
!pip install plotly

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS

plt.rc('figure',figsize=(17,13))
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
from plotly.subplots import make_subplots


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ms731/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 125 kB 755 kB/s eta 0:00:01
     |████████████████████████████████| 63 kB 864 kB/s eta 0:00:01
     |████████████████████████████████| 138 kB 800 kB/s eta 0:00:01
     |████████████████████████████████| 61 kB 782 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install twython

     |████████████████████████████████| 151 kB 750 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
data = pd.read_csv('Data_Cleaning/Individual_Tweets_4-29.csv')  

In [ ]:
data.head()

,Datetime,Tweet Id,Username,Like Count,Display Name,Text_lemmatized
0,2021-02-27 23:59:59+00:00,1365813964320501767,seeker375,0,Matthew,"['ive', 'trying', 'tweet', 'dr', 'yeadon', 'lo..."
1,2021-02-27 23:59:58+00:00,1365813959333318656,Richard31225984,0,Demosthenies MAN / MAN,"['food', 'drug', 'administration', 'authorized..."
2,2021-02-27 23:59:57+00:00,1365813958163111938,NxtLevelPromo_,36,NxtLevelPromo,"['ap', 'there', 'three', 'covid19', 'vaccine',..."
3,2021-02-27 23:59:49+00:00,1365813925082763266,WENYNicholasP,0,Nicholas Phillips,"['johnson', 'amp', 'johnson', 'covid19', 'vacc..."
4,2021-02-27 23:59:46+00:00,1365813911208030213,xphile101,1,Michelle,"['aamots20', 'likethe309', 'cousin', 'doctor',..."


In [ ]:
data.shape

(6024, 6)

In [ ]:
data['Text_lemmatized'] = data['Text_lemmatized'].str.replace('[^\w\s]','')

/var/folders/hx/0dxb0wn51wx5pscstjkwf2gc000qqf/T/ipykernel_42236/1596696550.py:1: FutureWarning:

The default value of regex will change from True to False in a future version.



In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
new_words = {
    'cpos': -3.0,
    'cneg': 3.0,
}
analyser = SentimentIntensityAnalyzer()
analyser.lexicon.update(new_words)
scores=[]
for i in range(len(data['Text_lemmatized'])):
    
    score = analyser.polarity_scores(data['Text_lemmatized'][i])
    score=score['compound']
    scores.append(score)
sentiment=[]
for i in scores:
    if i>=0.75:
        sentiment.append('Overly Positive')
    elif (i>=0.05) and (i<0.75):
        sentiment.append('Positive')
    elif i<=(-0.75):
        sentiment.append('Overly Negative')
    elif (i<=-0.05) and (i>-0.75):
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data['sentiment']= pd.Series(np.array(sentiment))

In [ ]:
data['score']= pd.Series(np.array(scores))

In [ ]:
data.head()

,Datetime,Tweet Id,Username,Like Count,Display Name,Text_lemmatized,sentiment,score
0,2021-02-27 23:59:59+00:00,1365813964320501767,seeker375,0,Matthew,ive trying tweet dr yeadon long time day would...,Negative,-0.4717
1,2021-02-27 23:59:58+00:00,1365813959333318656,Richard31225984,0,Demosthenies MAN / MAN,food drug administration authorized johnson am...,Negative,-0.6369
2,2021-02-27 23:59:57+00:00,1365813958163111938,NxtLevelPromo_,36,NxtLevelPromo,ap there three covid19 vaccine flint michigan ...,Negative,-0.3089
3,2021-02-27 23:59:49+00:00,1365813925082763266,WENYNicholasP,0,Nicholas Phillips,johnson amp johnson covid19 vaccine get emerge...,Negative,-0.3818
4,2021-02-27 23:59:46+00:00,1365813911208030213,xphile101,1,Michelle,aamots20 likethe309 cousin doctor covid last y...,Positive,0.6139


In [ ]:
data.shape

(6024, 8)

In [ ]:
data[data['cpos'] == True]

KeyError: 'cpos'

In [ ]:
data['Text_lemmatized'][2106225]

KeyError: 2106225

In [ ]:
data[data['cneg'] == True]

In [ ]:
data.groupby(by="sentiment").mean()

In [ ]:
temp = data.groupby('sentiment').count()['Text_lemmatized'].reset_index().sort_values(by='Text_lemmatized',ascending=False)
temp.style.background_gradient(cmap='Purples')

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='sentiment',data=data)
fig = go.Figure(go.Funnelarea(
    text =temp.sentiment,
    values = temp.Text_lemmatized,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

In [ ]:
data["Datetime"] = pd.to_datetime(data.Datetime) 
timeline = data.resample('D', on='Datetime')["sentiment"].value_counts().unstack(1)

timeline.reset_index(inplace=True)

timeline = timeline.melt("Datetime", var_name='sentiment',  value_name='vals')

sns.set_style("whitegrid")
sns.lineplot(x="Datetime", y="vals", hue="sentiment", data=timeline)
plt.figure(figsize=(6,4))

In [ ]:
data.to_csv('sentiment_analysis_final.csv', index = False)